In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

llm = ChatOpenAI(
  temperature=0.1,
  model="gpt-4o-mini",
)

class DuckDuckGoSearchToolArgSchma(BaseModel):
  query: str = Field(description="The search query to look up on DuckDuckGo.")

class DuckDuckGoSearchTool(BaseTool):
  name = "DuckDuckGoSearchTool"
  description = """
    Searches the web using DuckDuckGo and extracts content from the most relevant webpage.
  """

  args_schema: Type[DuckDuckGoSearchToolArgSchma] = DuckDuckGoSearchToolArgSchma

  def _run(self, query):
    ddg = DuckDuckGoSearchAPIWrapper()
    return ddg.run(query)

class WikipediaSearchToolArgSchma(BaseModel):
  query: str = Field(description="The search query to look up on Wikipedia.")

class WikipediaSearchTool(BaseTool):
  name = "WikipediaSearchTool"
  description = """
    Searches the web using Wikipedia and extracts content from the most relevant webpage.
  """

  args_schema: Type[WikipediaSearchToolArgSchma] = WikipediaSearchToolArgSchma

  def _run(self, query):
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)



In [18]:
agent = initialize_agent(
  llm=llm,
  verbose=True,
  agent=AgentType.OPENAI_FUNCTIONS,
  handle_parsing_errors=True,
  tools=[
    DuckDuckGoSearchTool(),
    WikipediaSearchTool(),
  ],
  agent_kwargs = {
    "system_message": SystemMessage(
        content="""
          You are an AI assistant that helps users by searching for webpages related to their query,
          extracting the main content from the most relevant result, and presenting it clearly.
          Respond in the same language the user used in their query. For example, if the user writes in Korean, reply in Korean.
        """
    )
  }
)


In [19]:
query = "the XZ backdoor에 대한 연구"
extract_context = agent.invoke(query)
extract_context



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils

{'input': 'the XZ backdoor에 대한 연구',
 'output': 'XZ 백도어에 대한 연구 결과는 다음과 같습니다.\n\n2024년 2월, "Jia Tan"이라는 이름을 사용하는 계정에 의해 XZ 유틸리티의 리눅스 빌드에 악성 백도어가 도입되었습니다. 이 백도어는 liblzma 라이브러리의 5.6.0 및 5.6.1 버전에서 발견되었으며, 특정 Ed448 개인 키를 가진 공격자가 영향을 받은 리눅스 시스템에서 OpenSSH를 통해 원격 코드 실행을 가능하게 합니다. 이 문제는 CVE-2024-3094라는 공통 취약점 및 노출 번호가 부여되었으며, CVSS 점수는 10.0으로, 이는 가능한 최고 점수입니다.\n\nXZ 유틸리티는 대부분의 리눅스 배포판에 일반적으로 존재하지만, 발견 당시 백도어가 포함된 버전은 아직 생산 시스템에 널리 배포되지 않았고, 주요 배포판의 개발 버전에서만 존재했습니다. 이 백도어는 소프트웨어 개발자인 Andres Freund에 의해 발견되었으며, 그는 2024년 3월 29일에 자신의 발견을 발표했습니다.\n\n이와 같은 백도어는 일반적으로 정상적인 인증이나 암호화를 우회하는 비밀스러운 방법으로 사용되며, 원격 접근을 확보하거나 암호 시스템에서 평문에 접근하는 데 사용됩니다.'}

In [25]:
file_path = f"./files/extract/{query}.txt"

with open(file_path, "w") as f:
  f.write(extract_context["output"])